In [122]:
# !pip install polars hvplot --upgrade
import polars as pl
import hvplot

def graph_me(df_, recording_id, width=1600, height=230, time_scale='minutes'):
    '''Takes a df and recording_id column (cumsum when new recording has smaller time value
    than preceding one) and outputs 3 graphs showing the magnitudes and force during recording'''
    display(df_
            .filter(pl.col('recording_id') == recording_id)
            .plot(x=time_scale, y='magnitude_of_direction_change', width=width, height=height,))
    display(df_
            .filter(pl.col('recording_id') == recording_id)
            .plot(x=time_scale, y='magnitude_of_acceleration_change', width=width, height=height,))
    display(df_
            .filter(pl.col('recording_id') == recording_id)
            .plot(x=time_scale, y='FSR_voltage', width=width, height=height,))
    # return df_


def pretty_my_datas(df_, time_scale='minutes'):
    '''Takes raw datalog file and formats things correctly, and identifies each new recording start as its
    own recording id'''
    return (df_
              .rename({'column_1':'time_ms', 'column_2':'BATvoltage', 'column_3':'AccelX', 'column_4':'AccelY', 'column_5':'AccelZ', 
                       'column_6':'GyroX', 'column_7':'GyroY', 'column_8':'GyroZ', 'column_9':'TemperatureC', 
                       'column_10':'FSR_voltage', 'column_11':'FSR_resistance', 'column_12':'Force'}
                           )
              .with_columns(TemperatureF = (pl.col('TemperatureC') * 1.8) + 32,
                            magnitude_of_acceleration_change = (pl.col('AccelX') * pl.col('AccelX') +
                                                                pl.col('AccelY') * pl.col('AccelY') +
                                                                pl.col('AccelZ') * pl.col('AccelZ')).sqrt(),
                            magnitude_of_direction_change = (pl.col('GyroX') * pl.col('GyroX') +
                                                            pl.col('GyroY') * pl.col('GyroY') +
                                                            pl.col('GyroZ') * pl.col('GyroZ')).sqrt(),
                            seconds = pl.col('time_ms')/1000,
                            minutes = pl.col('time_ms')/1000/60,
                           )
              #splits into groups based on following time value being lower
              .with_columns(recording_id = pl.when(pl.col('time_ms').shift() > pl.col('time_ms'))
                                            .then(pl.lit(1)))
              .with_columns(pl.col('recording_id').cum_sum().forward_fill().fill_null(0) + 1)
              #puts the recording id at the front
              .select(pl.col('recording_id'), 
                      pl.all().exclude('recording_id'))
             )

connection = pretty_my_datas(pl.read_csv('Night_DATALOG.TXT', has_header=False))
# connection = pretty_my_datas(pl.read_csv('2024-03-12_DATALOG.TXT', has_header=False))

connection

recording_id,time_ms,BATvoltage,AccelX,AccelY,AccelZ,GyroX,GyroY,GyroZ,TemperatureC,FSR_voltage,FSR_resistance,Force,TemperatureF,magnitude_of_acceleration_change,magnitude_of_direction_change,seconds,minutes
i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,1767,4.29,-0.01,3.46,-7.67,-0.24,-0.13,0.22,25.43,0.02,678.67,0.0,77.774,8.414309,0.350571,1.767,0.02945
1,1804,4.29,-0.48,4.72,-8.7,0.29,-0.02,-0.71,25.92,0.01,815.2,0.0,78.656,9.909531,0.767203,1.804,0.030067
1,1822,4.29,-0.22,5.36,-8.65,0.18,0.02,-0.12,25.97,0.01,815.2,0.0,78.746,10.178433,0.217256,1.822,0.030367
1,1839,4.29,-0.04,5.01,-8.6,0.2,-0.02,-0.08,25.98,0.02,581.14,0.0,78.764,9.952974,0.216333,1.839,0.03065
1,1857,4.29,-0.63,5.02,-8.53,0.32,-0.05,-0.06,26.02,0.02,678.67,0.0,78.836,9.91757,0.329393,1.857,0.03095
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
5,8727470,4.29,-1.32,-4.42,-7.8,0.11,-0.13,0.44,31.39,0.11,101.03,0.0,88.502,9.061942,0.471805,8727.47,145.457833
5,8727506,4.29,-0.27,-4.8,-7.86,0.14,-0.15,0.44,31.43,0.11,101.03,0.0,88.574,9.213713,0.485489,8727.506,145.458433
5,8727541,4.29,-1.16,-4.67,-7.18,0.15,-0.15,0.21,31.43,0.11,101.03,0.0,88.574,8.643315,0.298496,8727.541,145.459017


In [120]:
(connection
 # .filter(pl.col('minutes') > 0, pl.col('minutes') < 1.6)
 .pipe(graph_me, 3)
)

:Curve   [minutes]   (magnitude_of_direction_change)

:Curve   [minutes]   (magnitude_of_acceleration_change)

:Curve   [minutes]   (FSR_voltage)